## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

2024-03-08 16:39:19.815573: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-08 16:39:19.839345: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 16:39:20.177564: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Demo numpy based auto differentiation

In [2]:
import numpy as np

class Matmul:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x, W):
        h = np.matmul(x, W)
        self.mem={'x': x, 'W':W}
        return h
    
    def backward(self, grad_y):
        '''
        x: shape(N, d)
        w: shape(d, d')
        grad_y: shape(N, d')
        '''
        x = self.mem['x']
        W = self.mem['W']
        
        ####################
        '''计算矩阵乘法的对应的梯度'''
        ####################

        grad_x = np.matmul(grad_y, W.T) # output shape is (N, d)
        grad_W = np.matmul(x.T, grad_y) # output shape is (d, d')

        return grad_x, grad_W


class Relu:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x):
        self.mem['x']=x
        return np.where(x > 0, x, np.zeros_like(x))
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        ####################
        '''计算relu 激活函数对应的梯度'''
        ####################

        x = self.mem['x']
        grad_x = (x>0) * grad_y

        return grad_x
    


class Softmax:
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        x_exp = np.exp(x)
        partition = np.sum(x_exp, axis=1, keepdims=True)
        out = x_exp/(partition+self.epsilon)
        
        self.mem['out'] = out
        self.mem['x_exp'] = x_exp
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        s = self.mem['out']
        sisj = np.matmul(np.expand_dims(s,axis=2), np.expand_dims(s, axis=1)) # (N, c, c)
        g_y_exp = np.expand_dims(grad_y, axis=1)
        tmp = np.matmul(g_y_exp, sisj) #(N, 1, c)
        tmp = np.squeeze(tmp, axis=1)
        tmp = -tmp+grad_y*s 
        return tmp
    
class Log:
    '''
    softmax over last dimention
    '''
    def __init__(self):
        self.epsilon = 1e-12
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        out = np.log(x+self.epsilon)
        
        self.mem['x'] = x
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        x = self.mem['x']
        
        return 1./(x+1e-12) * grad_y
    


## Gradient check

In [3]:
import tensorflow as tf

x = np.random.normal(size=[5, 6])
W = np.random.normal(size=[6, 4])
aa = Matmul()
out = aa.forward(x, W) # shape(5, 4)
grad = aa.backward(np.ones_like(out))
print (grad)

with tf.GradientTape() as tape:
    x, W = tf.constant(x), tf.constant(W)
    tape.watch(x)
    y = tf.matmul(x, W)
    loss = tf.reduce_sum(y)
    grads = tape.gradient(loss, x)
    print (grads)

import tensorflow as tf

x = np.random.normal(size=[5, 6])
aa = Relu()
out = aa.forward(x) # shape(5, 4)
grad = aa.backward(np.ones_like(out))
print (grad)

with tf.GradientTape() as tape:
    x= tf.constant(x)
    tape.watch(x)
    y = tf.nn.relu(x)
    loss = tf.reduce_sum(y)
    grads = tape.gradient(loss, x)
    print (grads)

import tensorflow as tf
x = np.random.normal(size=[5, 6], scale=5.0, loc=1)
label = np.zeros_like(x)
label[0, 1]=1.
label[1, 0]=1
label[1, 1]=1
label[2, 3]=1
label[3, 5]=1
label[4, 0]=1
print(label)
aa = Softmax()
out = aa.forward(x) # shape(5, 6)
grad = aa.backward(label)
print (grad)

with tf.GradientTape() as tape:
    x= tf.constant(x)
    tape.watch(x)
    y = tf.nn.softmax(x)
    loss = tf.reduce_sum(y*label)
    grads = tape.gradient(loss, x)
    print (grads)

import tensorflow as tf

x = np.random.normal(size=[5, 6])
aa = Log()
out = aa.forward(x) # shape(5, 4)
grad = aa.backward(label)
print (grad)

with tf.GradientTape() as tape:
    x= tf.constant(x)
    tape.watch(x)
    y = tf.math.log(x)
    loss = tf.reduce_sum(y*label)
    grads = tape.gradient(loss, x)
    print (grads)

(array([[-1.23915438, -2.56477281, -1.29875307,  1.62557579, -0.18497403,
        -0.92279039],
       [-1.23915438, -2.56477281, -1.29875307,  1.62557579, -0.18497403,
        -0.92279039],
       [-1.23915438, -2.56477281, -1.29875307,  1.62557579, -0.18497403,
        -0.92279039],
       [-1.23915438, -2.56477281, -1.29875307,  1.62557579, -0.18497403,
        -0.92279039],
       [-1.23915438, -2.56477281, -1.29875307,  1.62557579, -0.18497403,
        -0.92279039]]), array([[-3.34600787, -3.34600787, -3.34600787, -3.34600787],
       [-0.96613349, -0.96613349, -0.96613349, -0.96613349],
       [ 0.56250529,  0.56250529,  0.56250529,  0.56250529],
       [-1.53420125, -1.53420125, -1.53420125, -1.53420125],
       [ 0.46955578,  0.46955578,  0.46955578,  0.46955578],
       [-4.04780866, -4.04780866, -4.04780866, -4.04780866]]))
tf.Tensor(
[[-1.23915438 -2.56477281 -1.29875307  1.62557579 -0.18497403 -0.92279039]
 [-1.23915438 -2.56477281 -1.29875307  1.62557579 -0.18497403 -0.922

2024-03-08 16:39:20.531567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-08 16:39:20.548775: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/tmp/ipykernel_42900/1929690988.py:98: RuntimeWarning: invalid value encountered in log
  out = np.log(x+self.epsilon)


# Final Gradient Check

In [4]:
import tensorflow as tf

label = np.zeros_like(x)
label[0, 1]=1.
label[1, 0]=1
label[2, 3]=1
label[3, 5]=1
label[4, 0]=1

x = np.random.normal(size=[5, 6])
W1 = np.random.normal(size=[6, 5])
W2 = np.random.normal(size=[5, 6])

mul_h1 = Matmul()
mul_h2 = Matmul()
relu = Relu()
softmax = Softmax()
log = Log()

h1 = mul_h1.forward(x, W1) # shape(5, 4)
h1_relu = relu.forward(h1)
h2 = mul_h2.forward(h1_relu, W2)
h2_soft = softmax.forward(h2)
h2_log = log.forward(h2_soft)


h2_log_grad = log.backward(label)
h2_soft_grad = softmax.backward(h2_log_grad)
h2_grad, W2_grad = mul_h2.backward(h2_soft_grad)
h1_relu_grad = relu.backward(h2_grad)
h1_grad, W1_grad = mul_h1.backward(h1_relu_grad)

print(h2_log_grad)
print('--'*20)
# print(W2_grad)

with tf.GradientTape() as tape:
    x, W1, W2, label = tf.constant(x), tf.constant(W1), tf.constant(W2), tf.constant(label)
    tape.watch(W1)
    tape.watch(W2)
    h1 = tf.matmul(x, W1)
    h1_relu = tf.nn.relu(h1)
    h2 = tf.matmul(h1_relu, W2)
    prob = tf.nn.softmax(h2)
    log_prob = tf.math.log(prob)
    loss = tf.reduce_sum(label * log_prob)
    grads = tape.gradient(loss, [prob])
    print (grads[0].numpy())

[[  0.         386.04462059   0.           0.           0.
    0.        ]
 [ 35.06772844   0.           0.           0.           0.
    0.        ]
 [  0.           0.           0.           1.31977073   0.
    0.        ]
 [  0.           0.           0.           0.           0.
   23.09438098]
 [ 11.82736681   0.           0.           0.           0.
    0.        ]]
----------------------------------------
[[  0.         386.04462074   0.           0.           0.
    0.        ]
 [ 35.06772844   0.           0.           0.           0.
    0.        ]
 [  0.           0.           0.           1.31977073   0.
    0.        ]
 [  0.           0.           0.           0.           0.
   23.09438098]
 [ 11.82736681   0.           0.           0.           0.
    0.        ]]


## 建立模型

In [5]:
class myModel:
    def __init__(self):
        
        self.W1 = np.random.normal(size=[28*28+1, 100])
        self.W2 = np.random.normal(size=[100, 10])
        
        self.mul_h1 = Matmul()
        self.mul_h2 = Matmul()
        self.relu = Relu()
        self.softmax = Softmax()
        self.log = Log()
        
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        bias = np.ones(shape=[x.shape[0], 1])
        x = np.concatenate([x, bias], axis=1)
        
        self.h1 = self.mul_h1.forward(x, self.W1) # shape(5, 4)
        self.h1_relu = self.relu.forward(self.h1)
        self.h2 = self.mul_h2.forward(self.h1_relu, self.W2)
        self.h2_soft = self.softmax.forward(self.h2)
        self.h2_log = self.log.forward(self.h2_soft)
            
    def backward(self, label):
        self.h2_log_grad = self.log.backward(-label)
        self.h2_soft_grad = self.softmax.backward(self.h2_log_grad)
        self.h2_grad, self.W2_grad = self.mul_h2.backward(self.h2_soft_grad)
        self.h1_relu_grad = self.relu.backward(self.h2_grad)
        self.h1_grad, self.W1_grad = self.mul_h1.backward(self.h1_relu_grad)
        
model = myModel()


## 计算 loss

In [6]:
def compute_loss(log_prob, labels):
     return np.mean(np.sum(-log_prob*labels, axis=1))
    

def compute_accuracy(log_prob, labels):
    predictions = np.argmax(log_prob, axis=1)
    truth = np.argmax(labels, axis=1)
    return np.mean(predictions==truth)

def train_one_step(model, x, y):
    model.forward(x)
    model.backward(y)
    model.W1 -= 1e-5* model.W1_grad
    model.W2 -= 1e-5* model.W2_grad
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

def test(model, x, y):
    model.forward(x)
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

## 实际训练

In [7]:
train_data, test_data = mnist_dataset()
train_label = np.zeros(shape=[train_data[0].shape[0], 10])
test_label = np.zeros(shape=[test_data[0].shape[0], 10])
train_label[np.arange(train_data[0].shape[0]), np.array(train_data[1])] = 1.
test_label[np.arange(test_data[0].shape[0]), np.array(test_data[1])] = 1.

for epoch in range(50):
    loss, accuracy = train_one_step(model, train_data[0], train_label)
    print('epoch', epoch, ': loss', loss, '; accuracy', accuracy)
loss, accuracy = test(model, test_data[0], test_label)

print('test loss', loss, '; accuracy', accuracy)

epoch 0 : loss 23.78640681612661 ; accuracy 0.10336666666666666
epoch 1 : loss 22.544828918528374 ; accuracy 0.1553
epoch 2 : loss 21.809606779845026 ; accuracy 0.17258333333333334
epoch 3 : loss 21.22756784601447 ; accuracy 0.19615
epoch 4 : loss 19.85035587573516 ; accuracy 0.24205
epoch 5 : loss 19.210006487949364 ; accuracy 0.2701
epoch 6 : loss 18.77535908922131 ; accuracy 0.2897166666666667
epoch 7 : loss 18.459241983648425 ; accuracy 0.3016666666666667
epoch 8 : loss 18.216926260760722 ; accuracy 0.3132666666666667
epoch 9 : loss 18.004642909291487 ; accuracy 0.31955
epoch 10 : loss 17.808560312882364 ; accuracy 0.3277
epoch 11 : loss 17.55944710494721 ; accuracy 0.3348
epoch 12 : loss 17.121411569575287 ; accuracy 0.34476666666666667
epoch 13 : loss 16.083595586508913 ; accuracy 0.36965
epoch 14 : loss 15.265418067582283 ; accuracy 0.4027
epoch 15 : loss 14.855865440853417 ; accuracy 0.42105
epoch 16 : loss 14.540155547807917 ; accuracy 0.43315
epoch 17 : loss 14.17106102463753